In [ ]:
import csv
import io
import json
import os
import tempfile
import zipfile
from collections import defaultdict
from datetime import datetime
from itertools import islice
from pathlib import Path

import pandas as pd
import py7zr
import requests
from dotenv import load_dotenv

# If you use GTFS Realtime protobufs, uncomment:
# from google.transit import gtfs_realtime_pb2

load_dotenv()

GTFS_REGIONAL_RT_KEY = os.getenv("GTFS_REGIONAL_RT_KEY")  # clé GTFS Regional Realtime
GTFS_REGIONAL_STATIC_KEY = os.getenv("GTFS_REGIONAL_STATIC_KEY")  # clé GTFS Regional Static